# BIG TODO: THE WHAT?

### How? With whom?

In [1]:
file = '../../data/papersummaries_cleaned.csv'

In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv(file)

### The Scheme

![](tagging_how.png)

NB: The Science category is deliberately not split into validation and evaluation. These terms are not very intuitive and respondents will hardly have realized any such distinction.

### The Rules

In [ ]:
def assign_tag(level_1, level_2, level_3, summary):
    tags = []
    for tag in level_2:
        if any([re.search(keyword, summary) for keyword in level_3[tag]]):
            tags.append(level_1+tag)
    return tags

In [321]:
def assign_engineering(summary):
    level_1 = ':engineering'
    
    level_2 = [':analysis', ':technology', ':method']
    
    level_3 = {':analysis':
                   ['A set of metrics']
               , 
               ':technology':
                   ['A tool', 'A solution', 'A model', 
                    'A taxonomy', 'An ontology', 'A (:?modell?ing |specification )language', 
                    'A template', '[Aa] blueprint', 'A (formal )?framework']
               , 
               ':method':
                   ['A method', 'A process', 'A.{,15}technique', 'training program']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)


def assign_science(summary):
    level_1 = ':science'
    
    level_2 = [':observation', ':intervention', ':interrogation']
    
    level_3 = {':observation':
                   ['(:?(:?multi.)?case|field) study',
                    '(:?data.|document.)driven study',
                    'industrial evaluation', 'An analysis']
               , 
               ':intervention':
                   ['experiment(?:s|\s)', 
                    'project-based study', 'workshop-based industrial study', 
                    'action research']
               ,
               ':interrogation':
                   ['interview-based study|study based on.{,30}interviews', 
                    'questionnaire', '(?<!literature )(?:online.)?survey']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)


def assign_perspective(summary):
    level_1 = ':perspective'
    
    level_2 = [':philosophy', ':opinion', ':experience', ':review']
    
    level_3 = {':philosophy':
                   ['conceptual framework']
               , 
               ':opinion':
                   ['A discussion', '\svision', 'roadmap\s']
               , 
               ':experience':
                   ['experience report']
               ,
               ':review':
                   ['literature (:?survey|study|review)',
                    'state of the art report']
               } 
    
    return assign_tag(level_1, level_2, level_3, summary)


def assign_all_how(summary):
    tags = ['how'+ x for x in 
            (assign_engineering(summary) 
             + assign_science(summary)
             + assign_perspective(summary))]
    return tags

In [329]:
def assign_all_withwhom(summary):
    level_1 = [':laypeople', ':professionals']
    level_2 = {':laypeople': [':students', ':others'],
               ':professionals': [':academics', ':practitioners']
              }
    level_3 = {':students': ['with students', 'with practitioners and students'],
               ':others': ['with crowd.?workers'],
               ':academics': ['with academics', 'with researchers', 
                              'with students and academics'],
               ':practitioners': ['with practitioners', 'with students and practitioners']
              }
    tags = []
    for l1 in level_1:
        for l2 in level_2[l1]:
            if any([re.search(x, summary) for x in level_3[l2]]):
                tags.append('withwhom'+l1+l2)
    return tags

NB: As of now, the third level isn't explicitly represented in the tags. That's unfortunate especially for the distinction experiment/survey (which I'd expect to be evaluated differently). I'd suggest we refine tags using a separate function (after the initial tag assignment).

In [322]:
df['how'] = [assign_all_how(x) for x in df.PaperSummary]

In [330]:
df['withwhom'] = [assign_all_withwhom(x) for x in df.PaperSummary]

In [331]:
df.head()

,PaperID,PaperSummary,how,withwhom
0,1,A method for automatically recovering software...,[how:engineering:method],[]
1,2,A set of two techniques for improving the qual...,[how:engineering:method],[]
2,3,A case study on evaluating a given technique f...,[how:science:observation],[]
3,4,An experience report on the development of a m...,[how:perspective:experience],[]
4,5,A document-driven study on the relevancy of cl...,[how:science:observation],[]


Summaries without tags?

In [236]:
[(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) < 1]

[]

Summaries with multiple tags? (to check whether they're okay)

In [237]:
[(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) > 1]

[(71,
  'A study based on experiments with students and a case study on the possible benefits of considering existing services and their alignment with requirements at a very early stage in order to exploit the desired benefits of reuse in service-oriented architectures',
  ['how:science:observation', 'how:science:intervention']),
 (207,
  'A literature survey on requirements elicitation techniques and a roadmap of research in order to improve the elicitation of tacit knowledge',
  ['how:perspective:opinion', 'how:perspective:review']),
 (368,
  'A set of two empirical studies (online survey to practitioners and experiment with students) on the creation and use of software requirement specifications in companies and the impact of their quality in subsequent development activities.',
  ['how:science:intervention', 'how:science:interrogation']),
 (395,
  'A literature study on specific threats to validity in controlled experiments with student participants and on mitigation strategies fo

In [464]:
longform_tags = pd.DataFrame(columns=['PaperID', 'Tag'])
for row in df.iterrows():
    longform_tags = longform_tags.append(
        pd.DataFrame({'PaperID':list(len(row[1][2])*[row[1][0]]), 'Tag':row[1][2]}))
    longform_tags = longform_tags.append(
        pd.DataFrame({'PaperID':list(len(row[1][3])*[row[1][0]]), 'Tag':row[1][3]}))
longform_tags = longform_tags.reset_index().drop('index', axis=1)
longform_tags.head()

,PaperID,Tag
0,1,how:engineering:method
1,2,how:engineering:method
2,3,how:science:observation
3,4,how:perspective:experience
4,5,how:science:observation


In [465]:
tags_three_levels = list(zip(*[x.split(':') for x in longform_tags.Tag]))
longform_tags['level_1'] = tags_three_levels[0]
longform_tags['level_2'] = tags_three_levels[1]
longform_tags['level_3'] = tags_three_levels[2]
longform_tags.head()

,PaperID,Tag,level_1,level_2,level_3
0,1,how:engineering:method,how,engineering,method
1,2,how:engineering:method,how,engineering,method
2,3,how:science:observation,how,science,observation
3,4,how:perspective:experience,how,perspective,experience
4,5,how:science:observation,how,science,observation


NB: some papers appear multiple times in the counts (here: twice at max) since some papers receive multiple tags.

In [466]:
longform_tags.groupby(['level_1', 'level_2', 'level_3']).count()[['PaperID']]

PaperID
level_1  level_2       level_3               
how      engineering   analysis             1
                       method             162
                       technology          52
         perspective   experience          38
                       opinion             11
                       philosophy           1
                       review              14
         science       interrogation       43
                       intervention        37
                       observation         81
withwhom laypeople     others               1
                       students            28
         professionals academics            2
                       practitioners       30

A tiny look at the usefulness of our preliminary tags...

In [467]:
evaluation_counts = pd.read_csv('../../analysis/20180325repract_evaluation_counts.csv')
smaller = evaluation_counts[list(evaluation_counts.columns.values[:5])+['TotalRatings']].rename({'PaperId':'PaperID'}, axis=1)

In [517]:
smaller.head(2)

,PaperID,Essential,Worthwhile,Unimportant,Unwise,TotalRatings
0,248,2,5,6,0,13
1,100,7,5,0,0,12


In [479]:
def level_1_wrangling(longformdf, evaldf, level_1_tag):
    result = longformdf[longformdf.level_1 == level_1_tag].merge(evaldf, on='PaperID'
                                                                ).groupby(['level_1', 'level_2', 'level_3',
                            'Essential', 'Worthwhile','Unimportant','Unwise', 'TotalRatings']).count(
    ).reset_index().groupby(['level_1', 'level_2', 'level_3']).sum(
    ).sort_values(['Essential','Worthwhile'],ascending=False).T[:-2].T
    for col in result.columns.values: 
        if col != 'TotalRatings':
            result[col] = result[col] / result['TotalRatings']
    result['PosPercentage'] = result['Essential'] + result['Worthwhile']
    return result.sort_values(['PosPercentage', 'Essential'], ascending=False)

In [480]:
level_1_wrangling(longform_tags, smaller, 'how')

Essential  Worthwhile  Unimportant  \
level_1 level_2     level_3                                             
how     science     observation     0.251515    0.475758     0.230303   
                    interrogation   0.315789    0.410526     0.236842   
        engineering method          0.303393    0.419162     0.215569   
                    analysis        0.200000    0.500000     0.200000   
                    technology      0.222222    0.462963     0.277778   
        perspective opinion         0.177778    0.488889     0.311111   
                    philosophy      0.166667    0.500000     0.166667   
                    experience      0.160494    0.487654     0.314815   
                    review          0.176471    0.441176     0.338235   
        science     intervention    0.112500    0.475000     0.343750   

                                     Unwise  TotalRatings  PosPercentage  
level_1 level_2     level_3                                               
how     science     observation    0.042424           330       0.727273  
                    interrogation  0.036842           190       0.726316  
        engineering method         0.061876           501       0.722555  
                    analysis       0.100000            10       0.700000  
                    technology     0.037037           216       0.685185  
        perspective opinion        0.022222            45       0.666667  
                    philosophy     0.166667             6       0.666667  
                    experience     0.037037           162       0.648148  
                    review         0.044118            68       0.617647  
        science     intervention   0.068750           160       0.587500

In [481]:
level_1_wrangling(longform_tags, smaller, 'withwhom')

Essential  Worthwhile  Unimportant  \
level_1  level_2       level_3                                             
withwhom professionals practitioners   0.319149    0.460993     0.205674   
         laypeople     students        0.090226    0.488722     0.353383   
         professionals academics       0.100000    0.400000     0.200000   
         laypeople     others          0.000000    0.000000     1.000000   

                                        Unwise  TotalRatings  PosPercentage  
level_1  level_2       level_3                                               
withwhom professionals practitioners  0.014184           141       0.780142  
         laypeople     students       0.067669           133       0.578947  
         professionals academics      0.300000            10       0.500000  
         laypeople     others         0.000000             1       0.000000

In [336]:
#longform_tags.merge(df[['PaperID', 'PaperSummary']]
#                   ).to_csv('../../analysis/papersummaries_tagged_how.csv', index=False)

The End.